<a href="https://colab.research.google.com/github/veryHapppy/study_ai/blob/main/Kaggle/regression_abalone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install optuna
!pip install catboost

In [ ]:
import numpy as np
import pandas as pd
import optuna
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from google.colab import drive
drive.mount('/content/drive')
import zipfile
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import root_mean_squared_error
from sklearn.ensemble import VotingRegressor, StackingRegressor
from sklearn.linear_model import RidgeCV
import joblib
from sklearn.compose import TransformedTargetRegressor

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def feature_engineering(data) :
  data = data[data['Height']>0]
  data['weight_rate'] = data['Whole weight.1'] / data['Whole weight']
  data['area'] = data['Length'] * data['Diameter']

  data = pd.get_dummies(data, columns=['Sex'])


  return data

In [ ]:
zip_path = '/content/drive/MyDrive/Colab Notebooks/[Regression]Regression with an Abalone Dataset/train.csv.zip'
extract_path = '/content/dataset/train'
with zipfile.ZipFile(zip_path, 'r') as z:
  z.extractall(extract_path)
train_set = pd.read_csv(f'{extract_path}/train.csv')

In [ ]:
train_set = train_set.set_index('id')
data = feature_engineering(train_set)
X = data.drop('Rings',axis=1)
y = data['Rings']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
y_train_log = np.log1p(y_train)
y_test_log = np.log1p(y_test)

/tmp/ipython-input-3285574806.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['weight_rate'] = data['Whole weight.1'] / data['Whole weight']
/tmp/ipython-input-3285574806.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['area'] = data['Length'] * data['Diameter']


In [ ]:
def objective(trial) :
  xgb_params = {
    'n_estimators': trial.suggest_int('xgb_n', 100, 500),
    'max_depth': trial.suggest_int('xgb_depth', 3, 7),
    'colsample_bytree': trial.suggest_float('xgb_colsample', 0.6, 0.8),
    'reg_alpha': trial.suggest_float('xgb_alpha', 1e-3, 10.0, log=True),
    'reg_lambda': trial.suggest_float('xgb_lambda', 1e-3, 10.0, log=True),
    'learning_rate': trial.suggest_float('xgb_lr', 0.01, 0.1),
  }
  lgbm_params = {
    'n_estimators': trial.suggest_int('lgbm_n', 100, 500),
    'num_leaves': trial.suggest_int('lgbm_leaves', 15, 50),
    'learning_rate': trial.suggest_float('lgbm_lr', 0.01, 0.1),
    'verbose': -1
  }
  cat_params = {
    'iterations': trial.suggest_int('cat_iter', 100, 500),
    'depth': trial.suggest_int('cat_depth', 3, 7),
    'learning_rate': trial.suggest_float('cat_lr', 0.01, 0.1),
    'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1.0, 10.0),
    'random_strength': trial.suggest_float('random_strength', 0.1, 10.0),
    'logging_level': 'Silent' # CatBoost 조용히 시키기
  }
  w_xgb = trial.suggest_float('w_xgb', 0.5, 2.0)
  w_lgbm = trial.suggest_float('w_lgbm', 0.5, 2.0)
  w_cat = trial.suggest_float('w_cat', 0.5, 2.0)

  xgb = XGBRegressor(**xgb_params, tree_method='hist', device='cuda', n_jobs=-1, random_state=42)
  lgbm = LGBMRegressor(**lgbm_params, device='gpu', n_jobs=-1, random_state=42)
  cat = CatBoostRegressor(**cat_params, task_type='GPU',devices='0', random_state=42)

  voting_model = VotingRegressor(
    estimators=[('xgb', xgb), ('lgbm', lgbm), ('cat', cat)],
    weights=[w_xgb, w_lgbm, w_cat],
  )

  tt_model = TransformedTargetRegressor(
    regressor=voting_model, # 아까 만든 보팅 모델
    func=np.log1p,
    check_inverse=False,
    inverse_func=np.expm1
  )

  score = cross_val_score(tt_model, X_train, y_train, cv=5, scoring='neg_root_mean_squared_error')

  return score.mean()

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=30)
print(-study.best_value)

[I 2026-01-23 20:22:31,342] A new study created in memory with name: no-name-089c2b16-b2b0-419e-a68a-54499455e1e5
[I 2026-01-23 20:23:11,779] Trial 0 finished with value: -1.8568841640097702 and parameters: {'xgb_n': 493, 'xgb_depth': 6, 'xgb_colsample': 0.7837162830736906, 'xgb_alpha': 0.9869492306270972, 'xgb_lambda': 0.0038556125495684736, 'xgb_lr': 0.05567260303604443, 'lgbm_n': 414, 'lgbm_leaves': 15, 'lgbm_lr': 0.07475389541570314, 'cat_iter': 435, 'cat_depth': 5, 'cat_lr': 0.01835496607982734, 'l2_leaf_reg': 2.73951075551399, 'random_strength': 1.5532589226115594, 'w_xgb': 0.7134669577714885, 'w_lgbm': 0.75954254625605, 'w_cat': 1.5791952749282991}. Best is trial 0 with value: -1.8568841640097702.
[I 2026-01-23 20:23:33,658] Trial 1 finished with value: -1.861759974581402 and parameters: {'xgb_n': 351, 'xgb_depth': 7, 'xgb_colsample': 0.6084259154789505, 'xgb_alpha': 2.5313502602278852, 'xgb_lambda': 0.7816515082811653, 'xgb_lr': 0.06934800167937002, 'lgbm_n': 261, 'lgbm_leaves'

1.8419957747220692


In [ ]:
from sklearn.utils.extmath import stable_cumsum
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

best = study.best_params

xgb_final = XGBRegressor(
    n_estimators=best['xgb_n'],
    max_depth=best['xgb_depth'],
    learning_rate=best['xgb_lr'],
    colsample_bytree=best['xgb_colsample'],
    reg_alpha=best['xgb_alpha'],
    reg_lambda=best['xgb_lambda'],
    random_state=42,
    tree_method='hist',
    device='cuda',
)

lgbm_final = LGBMRegressor(
    n_estimators=best['lgbm_n'],
    num_leaves=best['lgbm_leaves'],
    learning_rate=best['lgbm_lr'],
    random_state=42,
    verbose=-1,
    device='gpu',
)

cat_final = CatBoostRegressor(
    iterations=best['cat_iter'],
    depth=best['cat_depth'],
    learning_rate=best['cat_lr'],
    l2_leaf_reg=best['l2_leaf_reg'],
    random_strength=best['random_strength'],
    task_type='GPU', # GPU 사용
    devices='0',
    random_state=42,
    logging_level='Silent'
)

voting_model = VotingRegressor(
    estimators=[('xgb', xgb_final), ('lgbm', lgbm_final), ('cat', cat_final)],
    weights=[best['w_xgb'], best['w_lgbm'], best['w_cat']]
)

stacking_model = StackingRegressor(
    estimators=[('xgb', xgb_final), ('lgbm', lgbm_final), ('cat', cat_final)],
    final_estimator=RidgeCV(),
    cv=5
)

voting_model.fit(X_train, np.log1p(y_train))
stacking_model.fit(X_train, np.log1p(y_train))

voting_pred = np.expm1(voting_model.predict(X_test))
stacking_pred = np.expm1(stacking_model.predict(X_test))

print(f'voting 오차 : {root_mean_squared_error(y_test, voting_pred)}')
print(f'stacking 오차 : {root_mean_squared_error(y_test, stacking_pred)}')

joblib.dump(voting_model, 'voting_best.pkl')
joblib.dump(stacking_model, 'stacking_best.pkl')

voting 오차 : 1.845009558606474
stacking 오차 : 1.8418385984066104


['stacking_best.pkl']

In [ ]:
zip_path = '/content/drive/MyDrive/Colab Notebooks/[Regression]Regression with an Abalone Dataset/test.csv.zip'
extract_path = '/content/dataset/test'
with zipfile.ZipFile(zip_path, 'r') as z:
  z.extractall(extract_path)
test_set = pd.read_csv(f'{extract_path}/test.csv')

data = test_set.set_index('id')
data['weight_rate'] = data['Whole weight.1'] / data['Whole weight']
data['area'] = data['Length'] * data['Diameter']
data = pd.get_dummies(data, columns=['Sex'])

model = joblib.load('stacking_best.pkl')

prediction = np.expm1(model.predict(data))

submission = pd.DataFrame({
    "id": data.index,
    "Rings": prediction
})

submission.to_csv('submission.csv', index=False)
print("파일 생성 완료")

파일 생성 완료
